In [14]:
import requests
import re
import jieba

from lxml import etree
from bs4 import BeautifulSoup

In [13]:
"""
Author: Yan Jingcun
Date: 2022-04-27
Time: 08:10:23
Description: 简单爬取贴吧内容
"""


import requests
from lxml import etree
from bs4 import BeautifulSoup
import re


# url
url = 'https://tieba.baidu.com/f?ie=utf-8&kw=%E9%AC%BC%E8%B0%B7%E5%85%AB%E8%8D%92'

# 获取html文档
html = requests.get(url)

# 可以将从互联网上获取的源码数据加载到该对象中
pages = etree.HTML(html.content)

# xpath解析
# 选取文档中的所有元素, 找到类为 threadlist_title pull_left j_th_tit 的元素
# 选取其a标签, 提取a标签中的文本
xpath_title = '//*[@class="threadlist_title pull_left j_th_tit "]/a/text()'
xpath_time = '//*[@class="threadlist_reply_date pull_right j_reply_data"]/text()'
xpath_author = '//*[@class="frs-author-name j_user_card "]/text()'


# 使用xpath解析出标题
titles = pages.xpath(xpath_title)
times = pages.xpath(xpath_time)
authors = pages.xpath(xpath_author)

# 写入文档
i = 1
with open('titles.txt', 'w') as f:
  f.write('序号\t标题\t作者\t最后回复时间')
  for line, time, author in zip(titles, times, authors):
    f.write(str(i) + '\t' + line + '\t' + author + '\t' + time.strip() + '\n')
    i += 1

f.close()

print('SUCCESS')

SUCCESS


## 微信公众号文章爬取


In [ ]:
# -!- coding: utf-8 -!-

from selenium import webdriver
from selenium.webdriver.common.by import By


#加载chromedriver
driver = webdriver.Firefox(executable_path='./utils/geckodriver.exe', firefox_binary='U://FireFox/Firefox/firefox.exe')
#进入主页
url = 'http://weixin.sogou.com/'
driver.get(url)
#输入搜索内容
#Input = driver.find_element_by_xpath('//*[@id="query"]')
input = driver.find_element(By.XPATH,'//*[@id="query"]')
input.send_keys('三体')
#点击“搜文章”按钮
#button = driver.find_element_by_xpath('//*[@id="searchForm"]/div/input[3]')
button = driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/input[3]')
button.click()
#点击“搜索工具”按钮，显示高级搜素内容
#search = driver.find_element_by_xpath('//*[@id="tool_show"]/a')
search = driver.find_element(By.XPATH, '//*[@id="tool_show"]/a')
search.click()
#点击“全部时间”按钮，打开时间筛选框
#time = driver.find_element_by_xpath('//*[@id="time"]')
time = driver.find_element(By.XPATH, '//*[@id="time"]')
time.click()
#清空开始吋间，并输入开始时间
start_time = driver.find_element(By.XPATH, '//*[@id="date_start"]')
start_time.clear()
start_time.send_keys('2021-06-01')
#清空结朿时间，并输入结束时间
end_time = driver.find_element(By.XPATH, '//*[@id="date_end"]')
end_time.clear()
end_time.send_keys('2021-07-01')
#点击确定按钮
ok=driver.find_element(By.XPATH, '//*[@id="time_enter"]')
ok.click()
# link_list用于保存每个文章的url链接
link_list = list()
#遍历前10页的内容
for page in range(10):
#每一页有10篇文章
   for i in range(0, 9):
      try:
         #动态生成文章题目的xpath
         id ='//*[@id="sogou_vr_11002601_title_'+str(i)+ ' "]'
         # 使用get_attribute ()方法获取href属性值
         title = driver.find_element(By.XPATH, id).get_attribute('href')
         print(title)
         #把获取到的链接保存到link_list中去
         link_list.append(title)
      except:
         continue
#使用“下一页”按钮进行翻页
         next=driver.find_element(By.XPATH, '//*[@id="sogou_next"]')
      next.click()
#打开文件result2.txt，把获取的内容写入其中
with open('result2.txt', 'w', encoding='utf-8') as result:
    for link in link_list:
        try:
            #打开文章页面
            driver.get(link)
            #获取元素的文本内容，使用.text来获取
            title = driver.find_element(By.XPATH, '//*[@id="activity-name"]').text
            id = driver.find_element(By.XPATH, '//*[@id="post-user"]').text
            date = driver.find_element(By.XPATH, '//*[@id="post-date"]').text
            content = driver.find_element(By.XPATH, '//*[@id="js_content"]').text
            msg = date + '\t' + id+'\t' + title + '\n' +content + '\n' + '---'*60
            print(msg)
            #写入文件
            result.write(msg+'\n')
        except:
            continue
#关闭chromedriver
driver.close()


## 豆瓣评论爬取及词云展示

### 数据爬取

In [1]:
"""
Author: Yan Jingcun
Date: 2022-05-02
Time: 10:57:48
Description: 爬取数据并保存
"""

from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

bro = webdriver.Firefox(executable_path='./utils/geckodriver.exe', firefox_binary='U://FireFox/Firefox/firefox.exe')

url = 'https://book.douban.com/subject/5363767/reviews?start=20'

bro.get(url)
list_elements = bro.find_elements(by=By.CLASS_NAME, value="unfold")

# 这个页面一共的元素
element_number = len(list_elements)

with open('content2.txt', 'w+', encoding='utf-8') as f:

  # 打开每一个节点, 中间显式等待
  for i in range(element_number):
    list_elements = bro.find_elements(by=By.CLASS_NAME, value="unfold")[i].click()
    sleep(2)
    # 点击展开之后查找目标元素
    xpath = '//*[@class="review-content clearfix"]/text()'
    page_text = bro.page_source
    tree = etree.HTML(page_text)
    text = tree.xpath(xpath)
    for content in text:
      f.write(content)
    # 刷新页面
    bro.refresh()

bro.quit()

### 词云展示

In [29]:
from bs4 import BeautifulSoup
import jieba
import re
from wordcloud import WordCloud

#注意到获取的页面内容中有html的标签和其他符号存在，不利于下一步的词云操作，所以需要利用jieba和正则表达式对得到的文本进行清洗
r ='[，。\%、；1234567890n-】【“”]'

file=open("./content.txt","r",encoding='utf-8').read()

#剔除无关信息
file =re.sub(r,'',file)      
chinese_char = '[你我他她它和就是了都也这我们被一只那中个上与又所能到不还说可以很而但对为的在什么没有]'
file = re.sub(chinese_char,'',file)

#分词
con = jieba.lcut(file)     

#分词后插入空格
words = " ".join(con)    

#词云分析
wordcloud = WordCloud(font_path="simkai.ttf",background_color="white",width=1300, height=800).generate(words)

wordcloud.to_file('cloud.png')     
